In [1]:
pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 217.8 kB/s eta 0:00:0000:0100:02
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB


In [2]:
ward_data = pd.read_csv('/Users/jay/Desktop/DISS/wards_data.csv')
ward_data.head()

,NAME,ward_id,DISTRICT,LAGSSCODE,HECTARES,NONLD_AREA,geometry,ward name,borough,population,incident_count,demand_weight,centroid
0,Chessington South,E05000405,Kingston upon Thames,E09000021,755.173,0.0,POLYGON ((-0.3306790785982648 51.3290110240823...,Chessington South,Kingston upon Thames,10493,51,7882.50,POINT (-0.3120272800271884 51.34796572162466)
1,Tolworth and Hook Rise,E05000414,Kingston upon Thames,E09000021,259.464,0.0,POLYGON ((-0.2840948624066646 51.3651516247340...,Tolworth and Hook Rise,Kingston upon Thames,10294,57,7734.75,POINT (-0.2899047936994227 51.37426880136161)
2,Berrylands,E05000401,Kingston upon Thames,E09000021,145.390,0.0,POLYGON ((-0.3038496530100893 51.3924869780065...,Berrylands,Kingston upon Thames,9708,37,7290.25,POINT (-0.2894565229825078 51.39264588475685)
3,Alexandra,E05000400,Kingston upon Thames,E09000021,268.506,0.0,POLYGON ((-0.2725691349230317 51.3722653079621...,Alexandra,Kingston upon Thames,9639,30,7236.75,POINT (-0.2752732090845407 51.38388654659455)
4,Beverley,E05000402,Kingston upon Thames,E09000021,187.821,0.0,POLYGON ((-0.2587331652307956 51.4095594714878...,Beverley,Kingston upon Thames,10726,69,8061.75,POINT (-0.2589377056537231 51.40273959704683)


In [3]:
fire_stations = pd.read_csv('/Users/jay/Desktop/DISS/DISS_Data/Stations.csv')
fire_stations['station_id'] = fire_stations.index
fire_stations.head()

,name,address,borough,latitude,longitude,station_id
0,Dagenham,"Rainham Road North, Dagenham, Greater London R...",Barking and Dagenham,51.558785,0.157054,0
1,Barking,"Alfred's Way, Greater London, UK",Barking and Dagenham,51.532104,0.103288,1
2,Barnet,"144 Station Road, Barnet, Greater London EN5, UK",Barnet,51.646995,-0.185843,2
3,Mill Hill,"10 Hartley Avenue, Edgware, London, Greater Lo...",Barnet,51.615091,-0.243472,3
4,Finchley,"227 Long Lane, London Borough of Barnet, Londo...",Barnet,51.597836,-0.179377,4


In [4]:
# Demand locations and potential fire station sites
demand_locations = ward_data['ward_id'].tolist()
potential_fire_station_sites = fire_stations['station_id'].tolist()

# Demand weights
demand_weights = ward_data.set_index('ward_id')['demand_weight'].to_dict()

In [5]:
travel_times = pd.read_csv('/Users/jay/Desktop/DISS/DISS_Data/travel_times.csv')
travel_times.head()

,station_id,ward_id,travel_time
0,0,E05000410,4902
1,0,E05000285,3966
2,0,E05011481,3580
3,0,E05000111,2449
4,0,E05000342,4712


In [6]:
print(type(travel_times) )

<class 'pandas.core.frame.DataFrame'>


In [7]:
# Ensure the data is correctly formatted as DataFrames
if isinstance(travel_times, list):
    travel_times = pd.DataFrame(travel_times)

In [8]:
# Create a travel time matrix
travel_time_matrix = travel_times.pivot(index='ward_id', columns='station_id', values='travel_time').fillna(np.inf)

In [9]:
# Ensure all potential fire station sites are in the travel time matrix
travel_time_matrix = travel_time_matrix.loc[demand_locations, potential_fire_station_sites]


In [10]:
print("Travel Time Matrix:")
print(travel_time_matrix)

Travel Time Matrix:
station_id   0     1     2     3     4     5     6     7     8     9    ...  \
ward_id                                                                 ...   
E05000405   4104  3567  3932  3855  4011  3768  2976  3097  3453  3406  ...   
E05000414   4550  4014  3920  3878  3784  3541  3422  3542  3899  3178  ...   
E05000401   4908  4372  4401  3851  3757  3514  3780  3901  4257  3203  ...   
E05000400   4837  4301  4324  3774  3679  3437  3710  3830  4187  3074  ...   
E05000402   4912  4594  4274  3724  3630  3387  3784  3904  4261  3025  ...   
...          ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
E05000042    286   778  2708  3186  2661  3010  2609  2368  2533  3637  ...   
E05000030    269   862  2988  3467  2941  3291  2401  2161  2326  3918  ...   
E05000029    556  1115  2506  2984  2459  2808  2477  2237  2402  3436  ...   
E05000037    359   517  2643  3122  2597  2946  2356  2115  2280  3573  ...   
E05009297   2439  1781  3033  27

In [11]:
number_of_fire_stations = 102  #  number of fire stations to locate
response_time_standard = 360  # Response time standard in seconds (6 minutes according to LFB)

In [12]:
# Create a new model
model = gp.Model('MCLP')

Restricted license - for non-production use only - expires 2025-11-24


In [13]:
# Define decision variables
x = model.addVars(demand_locations, vtype=GRB.BINARY, name='X')
y = model.addVars(potential_fire_station_sites, vtype=GRB.BINARY, name='Y')


In [14]:
# Objective: Maximize the total weighted demand covered by selected fire stations
objective_1 = gp.quicksum(demand_weights[ward] * x[ward] for ward in demand_locations)


In [15]:
# Objective 2: Minimize the overlapping coverage of fire stations
C_j = {j: sum(1 for i in demand_locations if travel_time_matrix.at[i, j] <= 360) for j in potential_fire_station_sites}
objective_2 = gp.quicksum(C_j[j] * y[j] for j in potential_fire_station_sites) - gp.quicksum(x[i] for i in demand_locations)


In [16]:
# Objective 3: Minimize the total response time
#T_j = {j: travel_time_matrix.loc[:, j].sum() for j in potential_fire_station_sites}
#objective_3 = gp.quicksum(T_j[j] * y[j] for j in potential_fire_station_sites)


In [17]:
# Adding Objectives to the Model with Priorities
# Set objectives with hierarchical priorities
model.setObjectiveN(objective_1, index=0, priority=1, name="MaximizeWeightedDemandCoverage")  # Highest priority
model.setObjectiveN(objective_2, index=1, priority=2, name="MinimizeOverlappingCoverage")
#model.setObjectiveN(objective_3, index=2, priority=3, name="MinimizeResponseTime")


In [18]:
# Coverage Constraint: Ensure that if a demand location is covered, at least one of the fire stations within the response time standard must be selected
model.addConstrs((gp.quicksum(y[j] for j in potential_fire_station_sites if travel_time_matrix.at[ward, j] <= response_time_standard) >= x[ward] for ward in demand_locations), name="Coverage")


{'E05000405': <gurobi.Constr *Awaiting Model Update*>,
 'E05000414': <gurobi.Constr *Awaiting Model Update*>,
 'E05000401': <gurobi.Constr *Awaiting Model Update*>,
 'E05000400': <gurobi.Constr *Awaiting Model Update*>,
 'E05000402': <gurobi.Constr *Awaiting Model Update*>,
 'E05000406': <gurobi.Constr *Awaiting Model Update*>,
 'E05000404': <gurobi.Constr *Awaiting Model Update*>,
 'E05000413': <gurobi.Constr *Awaiting Model Update*>,
 'E05000410': <gurobi.Constr *Awaiting Model Update*>,
 'E05000412': <gurobi.Constr *Awaiting Model Update*>,
 'E05000408': <gurobi.Constr *Awaiting Model Update*>,
 'E05000403': <gurobi.Constr *Awaiting Model Update*>,
 'E05000409': <gurobi.Constr *Awaiting Model Update*>,
 'E05000407': <gurobi.Constr *Awaiting Model Update*>,
 'E05000411': <gurobi.Constr *Awaiting Model Update*>,
 'E05000415': <gurobi.Constr *Awaiting Model Update*>,
 'E05011466': <gurobi.Constr *Awaiting Model Update*>,
 'E05011476': <gurobi.Constr *Awaiting Model Update*>,
 'E0501148

In [19]:
# Total Number of Fire Stations: Ensure that exactly the specified number of fire stations are selected
model.addConstr(gp.quicksum(y[j] for j in potential_fire_station_sites) == number_of_fire_stations, name="TotalStations")


<gurobi.Constr *Awaiting Model Update*>

In [20]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 22.5.0 22F82)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 634 rows, 745 columns and 1115 nonzeros
Model fingerprint: 0x9d1d9796
Variable types: 0 continuous, 745 integer (745 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 297 rows and 297 columns
Presolve time: 0.01s
Presolved: 337 rows, 448 columns, 818 nonzeros
---------------------------

In [21]:
if model.status == GRB.OPTIMAL:
    selected_stations = [station for station in potential_fire_station_sites if y[station].x > 0.5]
    covered_wards = [ward for ward in demand_locations if x[ward].x > 0.5]
    
    print("Selected fire stations:", selected_stations)
    print("Covered demand locations:", covered_wards)
    
    # Calculate total covered demand
    total_covered_demand = sum(demand_weights[ward] for ward in covered_wards)
    print("Total covered demand:", total_covered_demand)
    
    # Debugging to ensure coverage
    for ward in demand_locations:
        if x[ward].x > 0.5:
            print(f"Ward {ward} is covered with weight {demand_weights[ward]}.")
        else:
            print(f"Ward {ward} is not covered.")
    
    # Debugging selected fire stations and their respective wards
    for station in selected_stations:
        covered_wards = [ward for ward in demand_locations if travel_time_matrix.at[ward, station] <= response_time_standard]
        print(f"Fire Station {station} covers wards: {covered_wards}")
    
    # Print the values of x[i]
    print("\nValues of x[i]:")
    for ward in demand_locations:
        print(f"x[{ward}] = {x[ward].x}")
else:
    print("No optimal solution found.")

Selected fire stations: [0, 1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 106, 107, 108, 109, 110, 111]
Covered demand locations: ['E05000401', 'E05000400', 'E05000402', 'E05000413', 'E05000410', 'E05000412', 'E05000408', 'E05000403', 'E05000409', 'E05000411', 'E05011466', 'E05011476', 'E05011487', 'E05011481', 'E05011480', 'E05011483', 'E05011465', 'E05011488', 'E05011472', 'E05011473', 'E05011471', 'E05011468', 'E05011475', 'E05011470', 'E05000107', 'E05000112', 'E05000120', 'E05000123', 'E05000113', 'E05000122', 'E05000348', 'E05000351', 'E05000362', 'E05000365', 'E05000172', 'E05000183', 'E05000182', 'E05000189', 'E05000192', 'E05000181', 'E05000184', 'E05

Fire Station 60 covers wards: ['E05000252', 'E05009397', 'E05009395', 'E05009388', 'E05009401', 'E05009390']
Fire Station 61 covers wards: ['E05009399', 'E05009404', 'E05009392', 'E05009396', 'E05000642']
Fire Station 62 covers wards: ['E05000408', 'E05000403', 'E05000409']
Fire Station 63 covers wards: ['E05000401', 'E05000413', 'E05000412', 'E05000408']
Fire Station 64 covers wards: ['E05000400', 'E05000402', 'E05000410', 'E05000411', 'E05000473', 'E05000469']
Fire Station 65 covers wards: ['E05000420', 'E05000421', 'E05000436', 'E05000423']
Fire Station 66 covers wards: ['E05000418', 'E05000425', 'E05000419', 'E05000623']
Fire Station 68 covers wards: ['E05000426', 'E05000427', 'E05000646']
Fire Station 69 covers wards: ['E05000424', 'E05000422', 'E05000434']
Fire Station 70 covers wards: ['E05000449', 'E05000443']
Fire Station 72 covers wards: ['E05011108', 'E05000453']
Fire Station 73 covers wards: ['E05000452', 'E05000444', 'E05000450', 'E05000441']
Fire Station 74 covers wards: 

In [22]:
selected_stations = [station for station in potential_fire_station_sites if y[station].x > 0.5]
covered_wards = [ward for ward in demand_locations if x[ward].x > 0.5]
    
print("Selected fire stations:", selected_stations)
print("Covered demand locations:", covered_wards)

Selected fire stations: [0, 1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 106, 107, 108, 109, 110, 111]
Covered demand locations: ['E05000401', 'E05000400', 'E05000402', 'E05000413', 'E05000410', 'E05000412', 'E05000408', 'E05000403', 'E05000409', 'E05000411', 'E05011466', 'E05011476', 'E05011487', 'E05011481', 'E05011480', 'E05011483', 'E05011465', 'E05011488', 'E05011472', 'E05011473', 'E05011471', 'E05011468', 'E05011475', 'E05011470', 'E05000107', 'E05000112', 'E05000120', 'E05000123', 'E05000113', 'E05000122', 'E05000348', 'E05000351', 'E05000362', 'E05000365', 'E05000172', 'E05000183', 'E05000182', 'E05000189', 'E05000192', 'E05000181', 'E05000184', 'E05

In [23]:
# Extract the deleted fire stations (not in selected_stations)
all_station_ids = fire_stations.index.tolist()
selected_stations = [station for station in all_station_ids if station in selected_stations]

# Filter the datasets based on the results
selected_stations_df = fire_stations.loc[selected_stations]
covered_wards_df = ward_data[ward_data['ward_id'].isin(covered_wards)]

# Save the filtered datasets
selected_stations_df.to_csv('selected_stations3.csv', index=False)
covered_wards_df.to_csv('covered_wards3.csv', index=False)


selected_stations_df.head(), covered_wards_df.head()

(        name                                            address  \
 0   Dagenham  Rainham Road North, Dagenham, Greater London R...   
 1    Barking                   Alfred's Way, Greater London, UK   
 2     Barnet   144 Station Road, Barnet, Greater London EN5, UK   
 3  Mill Hill  10 Hartley Avenue, Edgware, London, Greater Lo...   
 5     Hendon   91 The Burroughs, London, Greater London NW4, UK   
 
                 borough   latitude  longitude  station_id  
 0  Barking and Dagenham  51.558785   0.157054           0  
 1  Barking and Dagenham  51.532104   0.103288           1  
 2                Barnet  51.646995  -0.185843           2  
 3                Barnet  51.615091  -0.243472           3  
 5                Barnet  51.588719  -0.228379           5  ,
             NAME    ward_id              DISTRICT  LAGSSCODE  HECTARES  \
 2     Berrylands  E05000401  Kingston upon Thames  E09000021   145.390   
 3      Alexandra  E05000400  Kingston upon Thames  E09000021   268.506  

In [24]:
covered_wards_df

,NAME,ward_id,DISTRICT,LAGSSCODE,HECTARES,NONLD_AREA,geometry,ward name,borough,population,incident_count,demand_weight,centroid
2,Berrylands,E05000401,Kingston upon Thames,E09000021,145.390,0.0,POLYGON ((-0.3038496530100893 51.3924869780065...,Berrylands,Kingston upon Thames,9708,37,7290.25,POINT (-0.2894565229825078 51.39264588475685)
3,Alexandra,E05000400,Kingston upon Thames,E09000021,268.506,0.0,POLYGON ((-0.2725691349230317 51.3722653079621...,Alexandra,Kingston upon Thames,9639,30,7236.75,POINT (-0.2752732090845407 51.38388654659455)
4,Beverley,E05000402,Kingston upon Thames,E09000021,187.821,0.0,POLYGON ((-0.2587331652307956 51.4095594714878...,Beverley,Kingston upon Thames,10726,69,8061.75,POINT (-0.2589377056537231 51.40273959704683)
7,Surbiton Hill,E05000413,Kingston upon Thames,E09000021,166.482,0.0,POLYGON ((-0.313158566732084 51.38880045473526...,Surbiton Hill,Kingston upon Thames,11159,36,8378.25,POINT (-0.3013216518313546 51.38481088061287)
8,Old Malden,E05000410,Kingston upon Thames,E09000021,180.016,0.0,POLYGON ((-0.2611247640866895 51.3796003188058...,Old Malden,Kingston upon Thames,9749,24,7317.75,POINT (-0.255441922168587 51.38501354418145)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
626,Valence,E05000040,Barking and Dagenham,E09000002,129.284,0.0,POLYGON ((0.1423848152837226 51.55577648604088...,Valence,Barking and Dagenham,10617,43,7973.50,POINT (0.134158084752318 51.56057252049771)
627,Heath,E05000034,Barking and Dagenham,E09000002,200.128,0.0,POLYGON ((0.1542755823217178 51.56614239697808...,Heath,Barking and Dagenham,11629,88,8743.75,POINT (0.154339431869138 51.55752369745045)
628,Whalebone,E05000042,Barking and Dagenham,E09000002,155.399,0.0,POLYGON ((0.1542755823217178 51.56614239697808...,Whalebone,Barking and Dagenham,11410,46,8569.00,POINT (0.1421663395496752 51.5677448317706)
629,Eastbrook,E05000030,Barking and Dagenham,E09000002,345.060,0.0,"POLYGON ((0.161574053081384 51.54332646777141,...",Eastbrook,Barking and Dagenham,11348,47,8522.75,POINT (0.1735806979646452 51.55364754866076)
